In [1]:
%%capture
! curl -L https://git.io/GeoLite2-City.mmdb -o GeoLite2-City.mmdb 
! pip3 install geoip2
! pip3 install nslookup

In [2]:
%%capture
!wget -q https://ukraine-domains.domaintools.com/ukraine-domains-2022-03-{01..03}.txt 
!wget -q https://ukraine-domains.domaintools.com/ukraine-domains-2022-02-{25..28}.txt
!grep -h -v ^#  ukraine-domains-2022-03-{01..03}.txt >> domains.txt
!grep -h -v ^#  ukraine-domains-2022-02-{25..28}.txt >> domains.txt
!wc -l domains.txt
!sort -u -o domains.txt domains.txt
!wc -l domains.txt

In [3]:
import folium
from folium.plugins import MarkerCluster
import geoip2.database
import io
from nslookup import Nslookup
import os
import pandas as pd
import requests
import sys
import warnings
import zipfile

f = open(os.devnull, 'w')
sys.stderr = f
dns_query = Nslookup(dns_servers=["8.8.8.8"])

In [4]:
df = pd.read_csv ('domains.txt', names=['DNS'], header=None)
df['DNS'] = df['DNS'].astype('str')

In [5]:
df['ip_address'] = df['DNS'].map(lambda host:dns_query.dns_lookup(host).answer)
df["ip_address"] = df["ip_address"].str[0]
df = df.dropna()
df

,DNS,ip_address
0,1room4ukraine.com,146.59.209.152
1,200rfukraine.com,199.59.243.200
2,2forukraine.com,192.64.119.36
3,2getherforukraine.se,91.201.60.103
4,2pounds4ukraine.com,216.239.34.21
...,...,...
3585,youkraine-project.eu,81.169.145.70
3586,youkraine.tech,66.96.162.147
3587,yourhelptoukraine.com,34.102.136.180
3588,zaukraine.pl,168.119.212.196


In [6]:
def get_latitude(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.location.latitude
        return output
    except:
        return pd.np.nan


def get_longitude(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.location.longitude
        return output
    except:
        return pd.np.nan

def get_country(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = str(response.country.iso_code)
        return output
    except:
        return pd.np.nan

unique_ips = df['ip_address'].unique()
unique_ips = pd.Series(unique_ips, index = unique_ips)
df['Latitude'] = df['ip_address'].map(unique_ips.apply(get_latitude))
df['Longitude'] = df['ip_address'].map(unique_ips.apply(get_longitude))
df['Country'] = df['ip_address'].map(unique_ips.apply(get_country))
df = df.dropna()
df

,DNS,ip_address,Latitude,Longitude,Country
0,1room4ukraine.com,146.59.209.152,48.8582,2.3387,FR
1,200rfukraine.com,199.59.243.200,37.7510,-97.8220,US
2,2forukraine.com,192.64.119.36,37.7510,-97.8220,US
3,2getherforukraine.se,91.201.60.103,59.3247,18.0560,SE
4,2pounds4ukraine.com,216.239.34.21,34.0544,-118.2440,US
...,...,...,...,...,...
3585,youkraine-project.eu,81.169.145.70,51.2042,9.5717,DE
3586,youkraine.tech,66.96.162.147,37.7510,-97.8220,US
3587,yourhelptoukraine.com,34.102.136.180,39.1027,-94.5778,US
3588,zaukraine.pl,168.119.212.196,51.2993,9.4910,DE


In [7]:
df['Country'].value_counts().head(10)

US    1815
DE     388
CA     148
NL     126
CO      97
FR      81
PL      80
GB      72
UA      45
CZ      34
Name: Country, dtype: int64

In [9]:
df.to_csv('UA-Domains.csv', index=False)


In [8]:
map = folium.Map(location=[50, 0], tiles='cartodb positron', zoom_start=4)

mcluster = MarkerCluster().add_to(map)

for index, row in df.iterrows():
    folium.Marker(location=[row['Latitude'],row['Longitude']], popup=(row.DNS+"\n"+row.ip_address)).add_to(mcluster)

map
map.save("index.html")